In [ ]:
import pandas as pd

df = pd.read_excel(r"C:\uni\depi\rent_commercial3_uncleaned.xlsx", engine="openpyxl")
df

,location,area(m²),rent(EGP)
0,الاسكندرية / جناكليس,60,"15,000 ج.م"
1,الاسكندرية / شارع خليل حماده,90,"15,000 ج.م"
2,الاسكندرية / فيكتوريا,55,"25,000 ج.م"
3,الاسكندرية / شارع محمود العيسوي,75,"15,000 ج.م"
4,الاسكندرية / السيوف,250,"30,000 ج.م"
...,...,...,...
495,الاسكندرية / مرغم,1200,"135,000 ج.م"
496,الاسكندرية / شارع خليل المصري,65,"25,000 ج.م"
497,الاسكندرية / شارع مصطفي كامل,172,"80,000 ج.م"
498,الاسكندرية / شارع محمد فوزي معاذ,1000,"500,000 ج.م"


In [30]:
df["rent(EGP)"] = df["rent(EGP)"].str.strip()
df.loc[df["rent(EGP)"].str.contains(r"اتصل|لمعرفة|السعر", regex=True), "rent(EGP)"] = pd.NA
df["rent(EGP)"] = df["rent(EGP)"].str.replace(r"[^\d]", "", regex=True)
df["rent(EGP)"] = pd.to_numeric(df["rent(EGP)"], errors="coerce")
df.dropna(subset=["rent(EGP)"], inplace=True)

In [31]:
df

,location,area(m²),rent(EGP)
0,الاسكندرية / جناكليس,60,15000.0
1,الاسكندرية / شارع خليل حماده,90,15000.0
2,الاسكندرية / فيكتوريا,55,25000.0
3,الاسكندرية / شارع محمود العيسوي,75,15000.0
4,الاسكندرية / السيوف,250,30000.0
...,...,...,...
495,الاسكندرية / مرغم,1200,135000.0
496,الاسكندرية / شارع خليل المصري,65,25000.0
497,الاسكندرية / شارع مصطفي كامل,172,80000.0
498,الاسكندرية / شارع محمد فوزي معاذ,1000,500000.0


In [33]:
df.isnull().sum()

location     0
area(m²)     0
rent(EGP)    0
dtype: int64

In [ ]:
# translate location column to english
import re

df["location"] = df["location"].apply(lambda x: x.strip() if isinstance(x, str) else x)


AR_EN_MAP = {
    "الاسكندرية": "Alexandria",
    "الإسكندرية": "Alexandria",


    "شارع": "Street",
    "طريق": "Road",
    "ميدان": "Square",
    "كمبوند": "Compound",
    "شاطئ": "Beach",
    "بحري": "Bahary",
    "قبلي": "Qebly",
    "الصحراوي": "Desert",
    "طريق الحرية": "El-Horreya Road",
    "طريق الجيش": "El-Geish Road",
    "طريق قناة السويس": "Suez Canal Road",
    
    "جناكليس": "Gianaclis",
    "فيكتوريا": "Victoria",
    "السيوف": "El-Seyouf",
    "محرم بك": "Moharram Bek",
    "العامرية": "Amreya",
    "سيدي بشر": "Sidi Bishr",
    "سيدي بشر بحري": "Sidi Bishr Bahary",
    "سيدي بشر قبلي": "Sidi Bishr Qebly",
    "المعمورة": "El-Maamoura",
    "المعمورة الشاطئ": "El-Maamoura Beach",
    "بولكلي": "Bolkly",
    "العصافرة": "Asafra",
    "العصافرة بحري": "Asafra Bahary",
    "العصافرة قبلي": "Asafra Qebly",
    "البيطاش": "Beytash",
    "سموحة": "Smouha",
    "سموحة الجديدة": "New Smouha",
    "الإبراهيمية": "Ibrahimia",
    "باكوس": "Bakos",
    "جليم": "Gleem",
    "زيزينيا": "Zizinia",
    "سان ستيفانو": "San Stefano",
    "كامب شيزار": "Camp Caesar",
    "شدس": "Shods",
    "الهانوفيل": "Hannoville",
    "المنتزه": "Montaza",
    "سبورتنج": "Sporting",
    "كليوباترا": "Cleopatra",
    "الشاطبي": "Shatby",
    "أبيس": "Abis",
    "خورشيد": "Khorshid",
    "مرغم": "Merghem",
    "برج العرب": "Borg El Arab",
    "الكيلو 21": "Kilo 21",
    "الكيلو 26": "Kilo 26",
    "صواري": "Sawary",
    "ميامي": "Miami",
    "العجمي": "Agamy",
    "الورديان": "Wardian",
    "بيانكي": "Bianchi",
    "الرأس السوداء": "Ras El-Souda",
    "العوايد": "El-Awayed",
    "شرق": "Sharq",

    "شارع جمال عبد الناصر": "Gamal Abdel Nasser Street",
    "شارع محمد نجيب": "Mohamed Naguib Street",
    "شارع خالد بن الوليد": "Khaled Ibn Al-Walid Street",
    "شارع مصطفى كامل": "Mostafa Kamel Street",
    "شارع السلطان حسين": "Sultan Hussein Street",
    "شارع أبو قير - طريق الحرية": "Abu Qir Street - El-Horreya Road",
}

def normalize_arabic(s: str) -> str:
    """Light normalization so mapping hits more often."""
    s = s.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")
    s = s.replace("ى", "ي").replace("ة", "ه")
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip()

def translate_location_offline(text):
    if pd.isna(text):
        return pd.NA
    
    s = normalize_arabic(str(text).strip())
    
    for k in sorted(AR_EN_MAP.keys(), key=len, reverse=True):
        kn = normalize_arabic(k)
        s = s.replace(kn, AR_EN_MAP[k])
    
    s = re.sub(r"\s*/\s*", " / ", s)
    s = re.sub(r"\s*-\s*", " - ", s)
    s = re.sub(r"\s{2,}", " ", s).strip()
    return s


df["location"] = df["location"].apply(translate_location_offline)

In [ ]:
df.to_excel(r"C:\uni\depi\rent_commercial3_cleaned.xlsx", index=False)